# Forward Collars
Price forward collars using different models.

In [ ]:
from datetime import datetime
import numpy as np
import pyarrow as pa
import pandas as pd
from src.model.rbergomi import rBergomiMCModel
from data.samples import rbergomi_data, heston_data, localvol_data
from qablet.heston.mc import HestonMCModel
from qablet.black_scholes.mc import LVMCModel
from qablet_contracts.timetable import TS_EVENT_SCHEMA

### Forward Collars
Forward collars are not in the qablet_contracts package. We will write a method to create forward collars contracts here.

In [ ]:
# Create a Collar
def fwd_collar_timetable(ticker, strike_rate, width, strike_time, maturity):
    events = [
        {
            "track": None,
            "time": strike_time,
            "op": None,
            "quantity": 0,
            "unit": "FIX",  # set strike
        },
        {
            "track": "",
            "time": maturity,
            "op": "+",
            "quantity": 1,
            "unit": "COLLAR",  # pay collar
        },
    ]

    # Define the strike phrase, return the spot itself.
    def strike_fn(inputs):
        [s] = inputs
        return [s * strike_rate]

    # Define collar phrase.
    def collar_pay_fn(inputs):
        [s, k] = inputs

        ret = np.maximum(-width, s - k)
        ret = np.minimum(width, ret)

        return [ret]

    events_table = pa.RecordBatch.from_pylist(events, schema=TS_EVENT_SCHEMA)
    return {
        "events": events_table,
        "expressions": {
            "COLLAR": {
                "type": "phrase",
                "inp": [ticker, "K"],
                "fn": collar_pay_fn,
            },
            "FIX": {
                "type": "snapper",
                "inp": [ticker],
                "fn": strike_fn,
                "out": ["K"],
            },
        },
    }


# Try en example:
timetable = fwd_collar_timetable(
    "SPX", 1.0, 10, datetime(2013, 10, 31), datetime(2013, 12, 31)
)
print(timetable["events"].to_pandas())

  track                      time   op  quantity    unit
0   NaN 2013-10-31 00:00:00+00:00  NaN       0.0     FIX
1       2013-12-31 00:00:00+00:00    +       1.0  COLLAR


Create a series of forward collars starting at different expiration dates, setting strike one month prior.

In [ ]:
dates = pd.bdate_range(
    datetime(2013, 8, 30), datetime(2014, 8, 29), freq="2BME"
)

timetables = {}
for fix_dt, exp_dt in zip(dates[:-1], dates[1:]):
    timetables[exp_dt] = fwd_collar_timetable(
        "SPX", strike_rate=1.0, width=10, strike_time=fix_dt, maturity=exp_dt
    )

Bergomi Model

In [ ]:
model = rBergomiMCModel()
dataset = rbergomi_data()
dataset["MC"]["PATHS"] = 10_000  # very slow with 100_000, 1/250
dataset["MC"]["TIMESTEP"] = 1 / 100

for exp_dt, timetable in timetables.items():
    price, _ = model.price(timetable, dataset)
    print(f"{exp_dt.date()}: {price}")

2013-10-31: 1.164406689113938
2013-12-31: 1.1203102137754208
2014-02-28: 0.8551470647002714
2014-04-30: 0.9212186777743429
2014-06-30: 1.00874579200005
2014-08-29: 0.8135743113791019


Heston Model

In [ ]:
model = HestonMCModel()

for exp_dt, timetable in timetables.items():
    price, _ = model.price(timetable, heston_data())
    print(f"{exp_dt.date()}: {price}")

2013-10-31: 2.121521963757764
2013-12-31: 1.8388206132782627
2014-02-28: 1.9170084417735371
2014-04-30: 1.9957500899382765
2014-06-30: 1.8859125432198915
2014-08-29: 1.8048462874829085


Local Volatility MC Model

In [ ]:
model = LVMCModel()

for exp_dt, timetable in timetables.items():
    price, _ = model.price(timetable, localvol_data())
    print(f"{exp_dt.date()}: {price}")

2013-10-31: 2.090194178043658


2013-12-31: 1.4198356696442824
2014-02-28: 0.9136321273442298
2014-04-30: 0.571273888816799
2014-06-30: 0.3978848354095187
2014-08-29: 0.3226758088184244
